In [5]:
import altair as alt
import pandas as pd

#setting up data frame
df = pd.read_csv('Medals.csv')
df['NOC'] = df['NOC'].str.split('(').str[0].str.strip()
Medals = df[:154]


In [51]:
LF = 16
TF = 20

#setup for params/transformations
cntry = Medals['NOC'].unique()

brush_bar = alt.selection_interval()

single = alt.selection_point(on='mouseover')

dropdown = alt.binding_select(
    options=['Total', 'Gold', 'Silver', 'Bronze'],
    name='Medal (Bar Chart) '
)

ycol_param = alt.param(
    bind=dropdown,
    value='Total'  # Set default value
)

#creating the bar chart


# Creating the bar chart
bar = alt.Chart(Medals, title='Number of Selected Metals for Each Country').mark_bar().encode(
    x=alt.X('NOC:N', sort='-y', title='Country', axis=alt.Axis(labelFont='Book Antiqua', titleFont='Book Antiqua', labelFontSize=LF, titleFontSize=TF)),
    y=alt.Y('y:Q', title='Metals', axis=alt.Axis(labelFont='Book Antiqua', titleFont='Book Antiqua', labelFontSize=LF, titleFontSize=TF)),
    color=alt.condition(single, alt.value('DarkSlateBlue'), alt.value('lightgray'))
).transform_calculate(
    y=f'datum[{ycol_param.name}]'
).add_params(ycol_param, single)

text = bar.mark_text(
    align='center',
    baseline='middle',
    dy=-10,  # Adjust the position of the text
    color='DarkSlateBlue',
    fontSize=11,
    font='Book Antiqua',
    angle=25  # Rotate text slightly
).encode(
    text=alt.Text('y:Q')
)

# Layering the bar chart with text
final_bars = alt.layer(bar, text).interactive()

LF = 16
TF = 20

#create base to make scatters for each metal
base = alt.Chart(Medals, title = 'Amount of Metals for Each Country').mark_circle(size=60)

gld = base.encode(
    x=alt.X('NOC:N', title='Country', axis=alt.Axis(labelFont='Book Antiqua', titleFont='Book Antiqua', labelFontSize=LF, titleFontSize=TF)),
    y='Gold:Q', 
    color = alt.condition(brush_bar, alt.value('gold'), alt.value("dimgray")),
    tooltip = 'Gold'
)

svr = base.encode(
    x=alt.X('NOC:N', axis=alt.Axis(labelFont='Book Antiqua', titleFont='Book Antiqua', labelFontSize=LF, titleFontSize=TF)),    
    y='Silver:Q', 
    color = alt.condition(brush_bar, alt.value('lightgray'), alt.value("dimgray")),
    tooltip = 'Silver'
) + gld

brz = base.encode(
    x=alt.X('NOC:N', axis=alt.Axis(labelFont='Book Antiqua', titleFont='Book Antiqua', labelFontSize=LF, titleFontSize=TF)),
    y=alt.Y('Bronze:Q', axis=alt.Axis(labelFont='Book Antiqua', titleFont='Book Antiqua', labelFontSize=LF, titleFontSize=TF)),
    color = alt.condition(brush_bar, alt.value('orange'), alt.value("dimgray")),
    tooltip = 'Bronze'
) + svr


#show us the visualization
final_chart = final_bars.transform_filter(brush_bar) & brz.add_params(brush_bar)

final_chart.interactive()


c:\Users\owenb\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\owenb\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)

For my 5 interactive elements I chose to include zoom to closer look at data, a dropdown menu to select the type of metal, tooltips on the scatterplot to show the number of each metal, brushing and linking between my visuals to filter the countries in the bar graph, as well as mouseover actions for the bar graph to make the data easier to focus on. I chose a scatterplot in order to show the totals of each metal type per country, as with the long list of countries a stacked bar chart may get too dense. I chose the colors for the metals based on their real colors, so a legend would be clutter as those metal colors are universal common knowledge. Because of the large numbers, specific numbers are hard to differentiate between, so that is why I added the tooltips to my scatter as well as labels and zoom to the bars. My bar graph is sorted decending, and changes the types of metals being measured based on the dropdown and can filter based on brushing and linking. This is useful for answering nearly all the questions listed in the assignment, as you can highlight countries to filter, and can see the number of metals in the already-ordered bar chart
